## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [3]:
import nltk
nltk.download('stopwords')

import tensorflow as tf
tf.test.gpu_device_name()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


''

In [2]:
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing.text import one_hot
import numpy as np
import pandas as pd

In [6]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [ ]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [7]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [8]:
# Answer below:

yelp['normalized_text'] = [normalize(text) for text in yelp['text']]

yelp.head()

,text,sentiment,normalized_text
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [10]:
# Answer below:

vocab = " ".join(yelp['normalized_text'].tolist())
vocab = vocab.split(" ")
vocab = list(filter(None, vocab))
vocab_size = len(set(vocab))
vocab[:20]
len(set(vocab))

1628

In [11]:
vocab_size = len(set(vocab))
encoded_docs = [one_hot(doc, vocab_size) for doc in yelp['normalized_text']]

In [12]:
encoded_docs[:20]

[[341, 862, 993],
 [677, 958, 232],
 [958, 850, 146, 522],
 [1377, 1177, 1043, 1558, 265, 210, 344, 707, 862, 791],
 [335, 201, 1523, 607],
 [45, 5, 521, 1400, 703],
 [315, 1600, 1126],
 [1102, 598, 923, 1356, 805, 817, 297, 103, 1556, 934],
 [1605, 1523, 467],
 [1523, 532],
 [1380, 731],
 [1000, 958, 931, 412],
 [1306, 1280, 460, 1625, 132, 591, 388, 251, 1518],
 [360, 463, 1373, 85, 328, 13, 324],
 [402, 1249, 89, 137, 1396],
 [814, 1280, 593, 759, 342, 1545],
 [119, 707],
 [1527, 891, 1054, 1380],
 [993, 958, 466, 103, 646, 650, 181],
 [958, 598, 772]]

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [13]:
# Answer below:

from tensorflow.keras.preprocessing.sequence import pad_sequences

ind_vars = pad_sequences(encoded_docs)

ind_vars[:20]

array([[   0,    0,    0, ...,  341,  862,  993],
       [   0,    0,    0, ...,  677,  958,  232],
       [   0,    0,    0, ...,  850,  146,  522],
       ...,
       [   0,    0,    0, ...,  891, 1054, 1380],
       [   0,    0,    0, ...,  646,  650,  181],
       [   0,    0,    0, ...,  958,  598,  772]], dtype=int32)

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [14]:
# Answer below:

from sklearn.model_selection import train_test_split

# define X and y
X = ind_vars
y = yelp['sentiment']

# test/train split our data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
max_words = np.max(X_train)
max_words

1627

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [17]:
# Answer below:

import numpy as np
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

# instantiate model
model = Sequential()
# embedding layer
model.add(Embedding(input_dim=max_words+1, output_dim=100, input_length=X_train.shape[1]))
# simple RNN layer
model.add(SimpleRNN(32))
# dense layer
model.add(Dense(8, activation='relu'))
# dense output layer
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 83, 100)           162800    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                4256      
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 167,329
Trainable params: 167,329
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [18]:
# Answer below:

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128)

Epoch 1/50
7/7 [==============================] - 2s 89ms/step - loss: 0.6943 - accuracy: 0.5174 - val_loss: 0.6928 - val_accuracy: 0.4949
Epoch 2/50
7/7 [==============================] - 0s 40ms/step - loss: 0.6759 - accuracy: 0.5225 - val_loss: 0.6896 - val_accuracy: 0.5306
Epoch 3/50
7/7 [==============================] - 0s 42ms/step - loss: 0.6493 - accuracy: 0.6157 - val_loss: 0.6854 - val_accuracy: 0.5000
Epoch 4/50
7/7 [==============================] - 0s 38ms/step - loss: 0.6181 - accuracy: 0.5755 - val_loss: 0.6787 - val_accuracy: 0.5255
Epoch 5/50
7/7 [==============================] - 0s 37ms/step - loss: 0.5818 - accuracy: 0.6852 - val_loss: 0.6776 - val_accuracy: 0.5102
Epoch 6/50
7/7 [==============================] - 0s 39ms/step - loss: 0.5440 - accuracy: 0.7474 - val_loss: 0.6926 - val_accuracy: 0.5102
Epoch 7/50
7/7 [==============================] - 0s 37ms/step - loss: 0.4941 - accuracy: 0.8159 - val_loss: 0.6600 - val_accuracy: 0.5663
Epoch 8/50
7/7 [===========